# Transforms module demonstration

## Ladder operators and the canonical anticommutation relations

A system of $N$ fermionic modes is
described by a set of fermionic *annihilation operators*
$\{a_p\}_{p=0}^{N-1}$ satisfying the *canonical anticommutation relations*
$$\begin{aligned}
    \{a_p, a_q\} &= 0, \label{eq:car1} \\
    \{a_p, a^\dagger_q\} &= \delta_{pq}, \label{eq:car2}
  \end{aligned}$$ where $\{A, B\} := AB + BA$. The adjoint
$a^\dagger_p$ of an annihilation operator $a_p$ is called a *creation
operator*, and we refer to creation and annihilation operators as
fermionic *ladder operators*.
In a finite-dimensional vector space the anticommutation relations have the following consequences (see [here](http://michaelnielsen.org/blog/archive/notes/fermions_and_jordan_wigner.pdf) for a derivation and discussion):

-   The operators $\{a^\dagger_p a_p\}_{p=0}^{N-1}$ commute with each
    other and have eigenvalues 0 and 1. These are called the *occupation
    number operators*.

-   There is a normalized vector $\lvert{\text{vac}}\rangle$, called the *vacuum
    state*, which is a mutual 0-eigenvector of all
    the $a^\dagger_p a_p$.

-   If $\lvert{\psi}\rangle$ is a 0-eigenvector of $a_p^\dagger a_p$, then
    $a_p^\dagger\lvert{\psi}\rangle$ is a 1-eigenvector of $a_p^\dagger a_p$.
    This explains why we say that $a_p^\dagger$ creates a fermion in
    mode $p$.

-   If $\lvert{\psi}\rangle$ is a 1-eigenvector of $a_p^\dagger a_p$, then
    $a_p\lvert{\psi}\rangle$ is a 0-eigenvector of $a_p^\dagger a_p$. This
    explains why we say that $a_p$ annihilates a fermion in mode $p$.

-   $a_p^2 = 0$ for all $p$. One cannot create or annihilate a fermion
    in the same mode twice.

-   The set of $2^N$ vectors
    $$\lvert n_0 \ldots, n_{N-1} \rangle :=
      (a^\dagger_0)^{n_0} \cdots (a^\dagger_{N-1})^{n_{N-1}} \lvert{\text{vac}}\rangle,
      \qquad n_0, \ldots, n_{N-1} \in \{0, 1\}$$
    are orthonormal.

-   The annihilation operators $a_p$ act on this basis as follows:
    $$\begin{aligned}
        a_p \lvert n_0 \ldots, n_{p-1}, 1, n_{p+1}, n_{N-1} \rangle &=
        (-1)^{\sum_{q=0}^{p-1} n_q} \lvert n_0 \ldots, n_{p-1}, 0, n_{p+1}, n_{N-1} \rangle \\
        a_p \lvert n_0 \ldots, n_{p-1}, 0, n_{p+1}, n_{N-1} \rangle &= 0
      \end{aligned}$$

See [here](http://michaelnielsen.org/blog/archive/notes/fermions_and_jordan_wigner.pdf) for a derivation and discussion of these
consequences.

Let's instantiate some FermionOperators and check that they satisfy the canonical anticommutation relations. Let's also check that the occupation number operators commute.

In [1]:
from openfermion import *

# Create some ladder operators
a_2 = FermionOperator('2')
a_2_dag = FermionOperator('2^')
a_5 = FermionOperator('5')
a_5_dag = FermionOperator('5^')

# Construct occupation number operators
num_2 = a_2_dag * a_2
num_5 = a_5_dag * a_5

# Create FermionOperator versions of zero and identity
zero = FermionOperator()
identity = FermionOperator(())

# Check the canonical anticommutation relations
assert normal_ordered(anticommutator(a_2, a_2)) == zero
assert normal_ordered(anticommutator(a_2, a_2_dag)) == identity
assert normal_ordered(anticommutator(a_2, a_5)) == zero
assert normal_ordered(anticommutator(a_2, a_5_dag)) == zero

# Check that the occupation number operators commute
assert normal_ordered(commutator(num_2, num_5)) == zero

# Print some output
print('a_2 = \n{}'.format(a_2))
print('')
print('a_2_dag = \n{}'.format(a_2_dag))
print('')
print('a_5 = \n{}'.format(a_5))
print('')
print('a_5_dag = \n{}'.format(a_5_dag))
print('')
print('num_2 = \n{}'.format(num_2))
print('')
print('num_5 = \n{}'.format(num_5))

a_2 = 
1.0 [2]

a_2_dag = 
1.0 [2^]

a_5 = 
1.0 [5]

a_5_dag = 
1.0 [5^]

num_2 = 
1.0 [2^ 2]

num_5 = 
1.0 [5^ 5]


/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Mapping fermions to qubits with transforms

To simulate a system of fermions on a quantum computer, we must choose a representation of the ladder operators on the Hilbert space of the qubits. In other words, we must designate a set of qubit operators (matrices) which satisfy the canonical anticommutation relations. Qubit operators are written in terms of the Pauli matrices $X$, $Y$, and $Z$. In OpenFermion a representation is specified by a transform function which maps fermionic operators (typically instances of FermionOperator) to qubit operators (instances of QubitOperator). There are many choices for the representation.

### The Jordan-Wigner Transform
Under the Jordan-Wigner Transform (JWT), the annihilation operators are mapped to qubit operators as follows:
$$\begin{aligned}
    a_p &\mapsto \frac12 (X_p + \mathrm{i}Y_p) Z_1 \cdots Z_{p - 1} \\
    &= (\lvert{0}\rangle\langle{1}\rvert)_p Z_1 \cdots Z_{p - 1} \\
    &=: \tilde{a}_p.
\end{aligned}$$
This operator has the following action on a computational basis vector
$\lvert z_0, \ldots, z_{N-1} \rangle$:
$$\begin{aligned}
    \tilde{a}_p \lvert z_0 \ldots, z_{p-1}, 1, z_{p+1}, z_{N-1} \rangle &=
    (-1)^{\sum_{q=0}^{p-1} z_q} \lvert z_0 \ldots, z_{p-1}, 0, z_{p+1}, z_{N-1} \rangle \\
    \tilde{a}_p \lvert z_0 \ldots, z_{p-1}, 0, z_{p+1}, z_{N-1} \rangle &= 0.
  \end{aligned}$$
Note that $\lvert n_0 \ldots, n_{N-1} \rangle$ is a basis vector in the Hilbert space of fermions, while $\lvert z_0, \ldots, z_{N-1} \rangle$ is a basis vector in the Hilbert space of qubits. Similarly, in OpenFermion $a_p$ is a FermionOperator while $\tilde{a}_p$ is a QubitOperator.

By comparing the action of $\tilde{a}_p$ on $\lvert z_0, \ldots, z_{N-1} \rangle$ with the action of $a_p$ on $\lvert n_0 \ldots, n_{N-1} \rangle$, we can see that the JWT is associated with a particular mapping of bitstrings $e: \{0, 1\}^N \to \{0, 1\}^N$, namely, the identity map $e(x) = x$. In other words, under the JWT, the fermionic basis vector $\lvert n_0 \ldots, n_{N-1} \rangle$ is represented by the computational basis vector
$\lvert z_0=n_0, \ldots, z_{N-1}=n_{N-1} \rangle$.

Let's map our previously instantiated FermionOperators to QubitOperators using the JWT and check that the resulting operators satisfy the expected relations.

In [2]:
# Map FermionOperators to QubitOperators using the JWT
a_2_jw = jordan_wigner(a_2)
a_2_dag_jw = jordan_wigner(a_2_dag)
a_5_jw = jordan_wigner(a_5)
a_5_dag_jw = jordan_wigner(a_5_dag)

num_2_jw = jordan_wigner(num_2)
num_5_jw = jordan_wigner(num_5)

# Create QubitOperator versions of zero and identity
zero = QubitOperator()
identity = QubitOperator(())

# Check the canonical anticommutation relations
assert anticommutator(a_2_jw, a_2_jw) == zero
assert anticommutator(a_2_jw, a_2_dag_jw) == identity
assert anticommutator(a_2_jw, a_5_jw) == zero
assert anticommutator(a_2_jw, a_5_dag_jw) == zero

# Check that the occupation number operators commute
assert commutator(num_2_jw, num_5_jw) == zero

# Print some output
print("a_2_jw = \n{}".format(a_2_jw))
print('')
print("a_2_dag_jw = \n{}".format(a_2_dag_jw))
print('')
print("a_5_jw = \n{}".format(a_5_jw))
print('')
print("a_5_dag_jw = \n{}".format(a_5_dag_jw))
print('')
print("num_2_jw = \n{}".format(num_2_jw))
print('')
print("num_5_jw = \n{}".format(num_5_jw))

a_2_jw = 
0.5 [Z0 Z1 X2] +
0.5j [Z0 Z1 Y2]

a_2_dag_jw = 
0.5 [Z0 Z1 X2] +
-0.5j [Z0 Z1 Y2]

a_5_jw = 
0.5 [Z0 Z1 Z2 Z3 Z4 X5] +
0.5j [Z0 Z1 Z2 Z3 Z4 Y5]

a_5_dag_jw = 
0.5 [Z0 Z1 Z2 Z3 Z4 X5] +
-0.5j [Z0 Z1 Z2 Z3 Z4 Y5]

num_2_jw = 
(0.5+0j) [] +
(-0.5+0j) [Z2]

num_5_jw = 
(0.5+0j) [] +
(-0.5+0j) [Z5]


### The Bravyi-Kitaev transform

To define the Bravyi-Kitaev transform, it is easier to work with the operators

In [3]:
n_modes = 7

a_2_bk = bravyi_kitaev(a_2, n_modes)
a_2_dag_bk = bravyi_kitaev(a_2_dag, n_modes)
a_5_bk = bravyi_kitaev(a_5, n_modes)
a_5_dag_bk = bravyi_kitaev(a_5_dag, n_modes)

num_2_bk = bravyi_kitaev(num_2, n_modes)
num_5_bk = bravyi_kitaev(num_5, n_modes)

zero = QubitOperator()
identity = QubitOperator(())

# Check the canonical anticommutation relations
assert anticommutator(a_2_bk, a_2_bk) == zero
assert anticommutator(a_2_bk, a_2_dag_bk) == identity
assert anticommutator(a_2_bk, a_5_bk) == zero
assert anticommutator(a_2_bk, a_5_dag_bk) == zero

# Check that the occupation number operators commute
assert commutator(num_2_bk, num_5_bk) == zero

print('')
print("a_2_bk = \n{}".format(a_2_bk))
print('')
print("a_2_dag_bk = \n{}".format(a_2_dag_bk))
print('')
print("a_5_bk = \n{}".format(a_5_bk))
print('')
print("a_5_dag_bk = \n{}".format(a_5_dag_bk))
print('')
print("num_2_bk = \n{}".format(num_2_bk))
print('')
print("num_5_bk = \n{}".format(num_5_bk))


a_2_bk = 
0.5 [Z1 X2 X3 X6] +
0.5j [Z1 Y2 X3 X6]

a_2_dag_bk = 
0.5 [Z1 X2 X3 X6] +
-0.5j [Z1 Y2 X3 X6]

a_5_bk = 
0.5 [Z3 Z4 X5 X6] +
0.5j [Z3 Y5 X6]

a_5_dag_bk = 
0.5 [Z3 Z4 X5 X6] +
-0.5j [Z3 Y5 X6]

num_2_bk = 
(0.5+0j) [] +
(-0.5+0j) [Z2]

num_5_bk = 
(0.5+0j) [] +
(-0.5+0j) [Z4 Z5]


### The parity transform

$$\begin{aligned}
    a_p &\mapsto \frac12 (X_p Z_{p - 1} + \mathrm{i}Y_p) X_{p + 1} \cdots X_{N} \\
    &= \frac14 [(X_p + \mathrm{i} Y_p) (I + Z_{p - 1}) -
                (X_p - \mathrm{i} Y_p) (I - Z_{p - 1})]
               X_{p + 1} \cdots X_{N} \\
    &= [\parens{\ket{0}\bra{1}}_p \parens{\ket{0}\bra{0}}_{p - 1} -
        \parens{\ket{0}\bra{1}}_p \parens{\ket{1}\bra{1}}_{p - 1}]
       X_{p + 1} \cdots X_{N} \\
\end{aligned}$$

OpenFermion does not include a standalone function for performing the parity transform. However, there is functionality for defining new transforms specified by binary codes, and the binary code that specifies the parity transform is included in the library. See `examples/binary_code_transforms_demo.ipynb` for an explanation of this functionality.

In [17]:
n_modes = 7
def parity(fermion_operator):
    return binary_code_transform(fermion_operator, parity_code(n_modes))

print("parity(a_2) = \n{}".format(parity(a_2)))
print('')
print("parity(a_2_dag) = \n{}".format(parity(a_2_dag)))
print('')
print("parity(a_5) = \n{}".format(parity(a_5)))
print('')
print("parity(a_5_dag) = \n{}".format(parity(a_5_dag)))

parity(a_2) = 
0.5 [Z1 X2 X3 X4 X5 X6] +
(-0-0.5j) [Y2 X3 X4 X5 X6]

parity(a_2_dag) = 
0.5 [Z1 X2 X3 X4 X5 X6] +
0.5j [Y2 X3 X4 X5 X6]

parity(a_5) = 
0.5 [Z4 X5 X6] +
(-0-0.5j) [Y5 X6]

parity(a_5_dag) = 
0.5 [Z4 X5 X6] +
0.5j [Y5 X6]


$$\begin{aligned}
            \bra{\text{vac}} \bracks{\parens{a_N}^{j_N} \cdots \parens{a_1}^{j_1}}
            a_p 
            \bracks{\parens{a^\dagger_1}^{i_1} \cdots \parens{a^\dagger_N}^{i_N}} \ket{\text{vac}}
            =
            (-1)^{\sum_{q < p} i_q} \delta_{j_p, 0} \delta_{i_p, 1} \prod_{q \neq p} \delta_{j_q, i_q}.
          \end{aligned}$$